In [45]:
import json
import nltk
import inflect
import tqdm
import collections
engine = inflect.engine()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/luke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/luke/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
recipes = json.load(open("../data/1M/recipe1M_layers/layer1.json"))

In [36]:
recipes[2]

{'ingredients': [{'text': '8 tomatoes, quartered'},
  {'text': 'Kosher salt'},
  {'text': '1 red onion, cut into small dice'},
  {'text': '1 green bell pepper, cut into small dice'},
  {'text': '1 red bell pepper, cut into small dice'},
  {'text': '1 yellow bell pepper, cut into small dice'},
  {'text': '1/2 cucumber, cut into small dice'},
  {'text': 'Extra-virgin olive oil, for drizzling'},
  {'text': '3 leaves fresh basil, finely chopped'}],
 'url': 'http://www.foodnetwork.com/recipes/gazpacho1.html',
 'partition': 'train',
 'title': 'Gazpacho',
 'id': '000035f7ed',
 'instructions': [{'text': 'Add the tomatoes to a food processor with a pinch of salt and puree until smooth.'},
  {'text': 'Combine the onions, bell peppers and cucumbers with the tomato puree in a large bowl.'},
  {'text': 'Chill at least 1 hour.'},
  {'text': 'Drizzle with olive oil, garnish with chopped basil and serve.'}]}

In [23]:
det_ingr[2]

{'valid': [True, True, True, True, True, True, True, True, True],
 'id': '000035f7ed',
 'ingredients': [{'text': 'tomatoes'},
  {'text': 'kosher salt'},
  {'text': 'red onion'},
  {'text': 'green bell pepper'},
  {'text': 'red bell pepper'},
  {'text': 'yellow bell pepper'},
  {'text': 'cucumber'},
  {'text': 'olive oil'},
  {'text': 'fresh basil'}]}

In [56]:
def string_to_float(x) :
    x = x.split('/')
    if len(x) > 1 :            
        return (float(x[0])/float(x[1]))
    else :
        return float(x[0])
    
def string_to_frac(x) :
    if '/' in x:
        return string_to_float(x)
    else :
        return(float(x[0]) / float(x[1:]))
    
def singularize(x):
    ingr = engine.singular_noun(x)
    return x if (not ingr) else ingr

In [75]:
recipes[62]

{'ingredients': [{'text': '1 lb turkey, cut into bite size pieces'},
  {'text': '2 tablespoons Italian dressing'},
  {'text': '2 cups frozen mixed vegetables'},
  {'text': '1 (10 3/4 ounce) can98% fat free condensed cream of chicken soup'},
  {'text': '14 lb Velveeta cheese, cut into 1/2-inch cubes (or store brand)'},
  {'text': '1 sheet frozen puff pastry, thawed (1/2 of 17.3-oz. pkg.)'},
  {'text': '1 egg, beaten'}],
 'url': 'http://www.food.com/recipe/left-over-make-over-turkey-pot-pie-405443',
 'partition': 'train',
 'title': 'Left over Make over Turkey Pot Pie',
 'id': '0004a459ec',
 'instructions': [{'text': 'Prehear oven to 400 degrees F.'},
  {'text': 'Heat and stir turkey in dressing in large skillet on medium heat until warm.'},
  {'text': 'This can also be done in a microwave.'},
  {'text': 'Stir in vegetables and soup.'},
  {'text': '(I dice an onion and use frozen peas, carrots, lima beans, and white corn)'},
  {'text': 'Spoon into greased 9-inch square baking dish; top wi

In [76]:
det_ingr[411]

{'valid': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 'id': '001ab9a70d',
 'ingredients': [{'text': 'sausage links'},
  {'text': 'onion'},
  {'text': 'green bell pepper'},
  {'text': 'garlic cloves'},
  {'text': 'salt'},
  {'text': 'pepper'},
  {'text': 'dried thyme'},
  {'text': 'dried oregano'},
  {'text': 'ground beef'},
  {'text': 'beef broth'},
  {'text': 'tomato sauce'},
  {'text': 'soft breadcrumbs'},
  {'text': 'egg'},
  {'text': 'butter'},
  {'text': 'all - purpose flour'},
  {'text': 'beef broth'},
  {'text': 'tomato sauce'},
  {'text': 'salt'},
  {'text': 'pepper'}]}

In [ ]:
def parse_quantity(x) :
    if ((len(tags) >= 2) and (tags[0][1] == 'CD')) :
            
        #nb to nb unit ing
        if ((len(tags) > 4) and (tags[1][0] == 'to') and (tags[2][1] == 'CD')) :                
            unit_to_recognize = tags[3][0].lower()
            n1 = string_to_float(tags[0][0])
            n2 = string_to_float(tags[2][0])

        #nb nb unit ing
        elif tags[1][1] == 'CD' :
            unit_to_recognize = tags[2][0].lower()

        #nb -nb unit ing
        elif tags[1][0][0] == '-' :
            unit_to_recognize = tags[2][0].lower()

        #nb (nb unit) ing
        elif ((tags[1][0] == '(') and (len(tags)>6)):
            idx_par = tags.index((')', ')'))
            old_unit = tags[idx_par+1][0]
            unit_to_recognize = tags[idx_par-1][0].lower()

            quant = tags[2:idx_par]
            ingr = tags[idx_par+1:]
            tags = quant+ingr

        else :
            unit_to_recognize = tags[1][0].lower()


In [67]:
units = ["bushel", "cup", "dash", "drop", "fl." "oz", "g", "gallon", "glass",
"kg", "liter", "ml", "ounce", "c.", "pinch", "pint", "pound", "lb", "quart",
"scoop", "shot", "tablespoon", "teaspoon", "tsp", "tbsp"]


measurable_indices = []
unit_ing = collections.Counter()
measurables = 0
nb_to_try = 100

for e, r in tqdm.tqdm_notebook(enumerate(recipes[:nb_to_try])) :
    
    contains_immesurable = False
    quantities = []
    
    for i, ingredient in enumerate(r['ingredients']) :
        tags = nltk.pos_tag(nltk.word_tokenize(ingredient['text']))
        n1 = 0
        n2 = 0
        
        if ((len(tags) > 2) and (tags[0][1] == 'CD')) :
            
            #nb to nb unit ing
            if ((len(tags) > 4) and (tags[1][0] == 'to') and (tags[2][1] == 'CD')) :                
                unit_to_recognize = tags[3][0].lower()
                n1 = string_to_float(tags[0][0])
                n2 = string_to_float(tags[2][0])
                 
            #nb nb unit ing
            elif tags[1][1] == 'CD' :
                unit_to_recognize = tags[2][0].lower()
                
            #nb -nb unit ing
            elif tags[1][0][0] == '-' :
                unit_to_recognize = tags[2][0].lower()
                
            #nb (nb unit) ing
            elif ((tags[1][0] == '(') and (len(tags)>6)):
                idx_par = tags.index((')', ')'))
                old_unit = tags[idx_par+1][0]
                unit_to_recognize = tags[idx_par-1][0].lower()
                
                quant = tags[2:idx_par]
                ingr = tags[idx_par+1:]
                tags = quant+ingr
                
            else :
                unit_to_recognize = tags[1][0].lower()
                
                
            detected =  det_ingr[e]['ingredients'][i]['text']
            if tags[1][0] == detected.split(" ")[0] :
                unit_ing.update([singularize(detected.split(" ")[0])])
                    
               
                
            singularized = engine.singular_noun(unit_to_recognize)            
            if singularized != False :
                unit_to_recognize = singularized
                       
            
            
            if (not unit_to_recognize in units) :
                contains_immesurable = True
                break

    if not contains_immesurable :
        measurables += 1
        measurable_indices.append(e)
        
print(str(measurables) + " recipes contained only mesurable ingredients => " + str(100*measurables/nb_to_try) + "%")
#json.dump(measurable_indices, open("./../generated/measurable_1M_recipes.json", 'w'))
                

38
46
62
99

26 recipes contained only mesurable ingredients => 26.0%


In [64]:
unit_ing.most_common()

[('egg', 397),
 ('garlic', 327),
 ('onion', 143),
 ('boneles', 101),
 ('green', 100),
 ('chicken', 83),
 ('red', 60),
 ('lemon', 50),
 ('whole', 38),
 ('bay', 37),
 ('carrot', 31),
 ('banana', 25),
 ('apple', 23),
 ('scallion', 23),
 ('pork', 22),
 ('vanilla', 22),
 ('dried', 22),
 ('shallot', 21),
 ('fresh', 21),
 ('orange', 20),
 ('lime', 18),
 ('yellow', 15),
 ('cinnamon', 15),
 ('sweet', 14),
 ('jalapeno', 14),
 ('tomato', 13),
 ('avocado', 13),
 ('salmon', 12),
 ('leek', 11),
 ('celery', 11),
 ('graham', 10),
 ('potato', 10),
 ('lamb', 9),
 ('flour', 9),
 ('zucchini', 9),
 ('beef', 9),
 ('poblano', 8),
 ('English', 8),
 ('pie', 8),
 ('pineapple', 8),
 ('ice', 8),
 ('hot', 8),
 ('corn', 7),
 ('plum', 7),
 ('chipotle', 7),
 ('minced', 7),
 ('white', 6),
 ('bacon', 6),
 ('strawberry', 6),
 ('pita', 6),
 ('cucumber', 6),
 ('maraschino', 6),
 ('bell', 6),
 ('Granny', 6),
 ('skinles', 6),
 ('Italian', 5),
 ('wonton', 5),
 ('baking', 5),
 ('cube', 5),
 ('frozen', 5),
 ('cherry', 5),
 ('k

In [4]:
det_ingr = json.load(open("../data/1M/det_ingr.json"))

In [5]:
det_ingr[0]

{'valid': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  False],
 'id': '000018c8a5',
 'ingredients': [{'text': 'penne'},
  {'text': 'cheese sauce'},
  {'text': 'cheddar cheese'},
  {'text': 'gruyere cheese'},
  {'text': 'dried chipotle powder'},
  {'text': 'unsalted butter'},
  {'text': 'all - purpose flour'},
  {'text': 'milk'},
  {'text': '14 ounces semihard cheese (page 23), grated (about 3 1/2 cups)'},
  {'text': '2 ounces semisoft cheese (page 23), grated (1/2 cup)'},
  {'text': 'kosher salt'},
  {'text': 'dried chipotle powder'},
  {'text': 'garlic powder'},
  {'text': '(makes about 4 cups)'}]}

In [83]:
usda_mappable_indices = []
usda_mappable = 0

for index in tqdm.tqdm_notebook(measurable_indices) :
    try :
        det_ingr[index]['valid'].index(False)
    except :
        usda_mappable += 1
        usda_mappable_indices.append(index)

print(str(usda_mappable) + " recipes contained only mappable ingredients => " + str(100*usda_mappable/len(measurable_indices)) + "%")


107977 recipes contained only mappable ingredients => 93.23875068001071%
